In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("2017_2019.csv")
df.drop(['Unnamed: 18'],inplace=True,axis=1)

In [ ]:
dates = set()
for i,data in df.iterrows():
    dates.add(pd.to_datetime( 
                             str(data['Year'])[:-2]+'-'+str(data['Month'])[:-2] + '-' + str(data['Day'])[:-2],
                             format="%Y-%m-%d"
                             ))

In [ ]:
def get_day(df:pd.DataFrame,day:pd.Timestamp) -> pd.DataFrame:
    
    return df.loc[ (df ['Year'] == day.year) & 
                  (df['Day'] == day.day) &
                  (df['Month'] == day.month)]
    
def get_records_night(df:pd.DataFrame,day:pd.Timestamp):
    records_beafore_df = get_n_days_beafore(df,day,1)
    records_beafore_df = records_beafore_df.loc[(records_beafore_df.Hour > 12)  & (records_beafore_df["Clearsky DNI"] != 0)]
    
    records_df = get_day(df,day)
    records_df = records_df.loc[(records_df.Hour < 12) &( records_df.DNI == 0) &(records_df["Clearsky DNI"] != 0)]
    
    return(pd.concat([records_beafore_df,records_df]))
    
    
  
def get_n_days_beafore(df:pd.DataFrame,day:pd.Timestamp,n:int):
    day_beafore = day - pd.offsets.Day(n)
    return get_day(df,day_beafore)

def get_working_day(df:pd.DataFrame,day:pd.Timestamp):
    records = get_day(df,day)
    return records.loc[(records.DNI != 0) | (records["Clearsky DNI"] != 0)]

    
    

In [ ]:
_ = pd.Series( list(dates)).sort_values()
get_records_night(df,_.iloc[2])

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
records_df =pd.DataFrame()
for idx,date in enumerate( pd.Series( list(dates)).sort_values().values):
    
    date = pd.to_datetime(date)
    day = get_working_day(df,date)
    record = pd.DataFrame([{'date':date}])
    
    record['month'] = date.month
    record['len_day'] = len(day)
    record['temp_mean'] = day.Temperature.mean()
    record['press_mean'] = day.Pressure.mean()
    record['wind_mean'] = day["Wind Speed"].mean()
    
    
    night = get_records_night(df,date)
    
    if idx == 0:
        record['len_night'] = 0
    else:
        record['len_night'] = len(night)
        record['night_temp_mean'] = night.Temperature.mean()
        record['night_press_mean'] = night.Pressure.mean()
        record['night_wind_mean'] = night["Wind Speed"].mean()
    
    record['Y'] = (day["Clearsky DNI"].sum() + day['DNI'].sum()) / record['len_day']
    records_df = pd.concat([records_df,record])
records_df

In [ ]:
records_df[records_df.isna().any(axis=1)]

In [ ]:
records_df.dropna(inplace=True)
records_df.drop(["date"], inplace= True,axis= 1)

In [ ]:
def get_Data_n_records(df, num, flatten_x=False):
    X = []
    Y = []
    for i in range(len(df)):
        if i < num + 1:
            pass
        else:
            X.append(df.iloc[i-num-1:i-1].values)
            Y.append(df.iloc[i].Y)
    
    if flatten_x:  # Option to return one long row containing the X data
        X = np.array(X)
        first, second, third = X.shape
        return  X.reshape(first, second * third), np.array(Y)

    
    return np.array(X), np.array(Y)

X,Y = get_Data_n_records(records_df, 3)

In [ ]:
TESTED_RECORD = 1

X, _ = get_Data_n_records(records_df, TESTED_RECORD)
X = np.array(X)
first, second, third = X.shape
X = X.reshape(first, second * third)

X_flattened, _ = get_Data_n_records(records_df, TESTED_RECORD, flatten_x=True)
assert X.shape == X_flattened.shape

for i in range(len(X)):
    if sum([X[i, x] != X_flattened[i, x] for x in range(X.shape[1])]) >= 1:
        control = [X[i, x] != X_flattened[i, x] for x in range(X.shape[1])]
assert (X == X_flattened).all()


assert pd.Series(X_flattened[1]).sum() == pd.Series(X[1]).sum()

In [ ]:
f1 = pd.DataFrame(X_flattened)#[:, control])
f1.head()

In [ ]:
f2 = pd.DataFrame(X)
f2.head()

In [ ]:
# import seaborn as sns
# sns.heatmap(f2 == f1)
# (f2 == f1)

In [ ]:
!pip install keras

In [ ]:
!conda install tensorflow

In [ ]:
# import pandas as pd
# import numpy as np 
# import keras as kr
# import sklearn as skl
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# import tensorflow as tf

In [ ]:
X, Y = get_Data_n_records(records_df,3, flatten_x=True)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y)

# regr = RandomForestRegressor(random_state=0)
# regr.fit(x_train, y_train)

# mlp_model = kr.Sequential([
#     kr.layers.Flatten(input_shape = (3,10)),
#     kr.layers.Dense(128, activation = 'relu'),
#     kr.layers.BatchNormalization(),
#     kr.layers.Dropout(0.1),
#     kr.layers.Dense(64, activation='relu'),
#     kr.layers.BatchNormalization(),
#     kr.layers.Dropout(0.1),
#     kr.layers.Dense(1, activation = 'sigmoid')
# ])

# mlp_model.compile()
# history = mlp_model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

# Random Forest GridSearch

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

n_estimators =[4]#  [4, 20, 100, 500, 2000] # Number of trees in the forest
max_depth = [2]# [ 7, 15] # Maximum number of levels in the tree
max_features = ['sqrt', 'auto'] # Number of features to consider at every split
min_samples_split = [2]#, 3, 5, 7] # Minimum number of samples required to split a node
min_samples_leaf = [2]#, 3] # Minimum number of samples required at each leaf node
bootstrap = [True, False] # Method of selecting samples for training each tree

param_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

forest = RandomForestRegressor(random_state=0)
forest_grid = GridSearchCV(estimator=forest,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3, verbose=2, n_jobs=-1,error_score="raise")

In [ ]:
forest_grid.fit(x_train, y_train)